In [64]:
import numpy as np
import pandas as pd
from nba_api.stats.endpoints import teamdashlineups, shotchartlineupdetail
from nba_api.stats.static import teams
import time

In [60]:
def get_all_team_lineups(team_id, season="2023-24"):
    print("Collecting lineups for team_id: " + str(team_id))
    data = teamdashlineups.TeamDashLineups(team_id=team_id)
    data = data.get_data_frames()[1]
    
    lineup_ids = data["GROUP_ID"].tolist()
    player_one = []
    player_two = []
    player_three = []
    player_four = []
    player_five = []
    mins = data["MIN"].tolist()
    
    for lineup_id in lineup_ids:
        players = lineup_id.split("-")[1:-1]
        player_one.append(players[0])
        player_two.append(players[1])
        player_three.append(players[2])
        player_four.append(players[3])
        player_five.append(players[4])
        
    out = {
        "LINEUP_ID": lineup_ids,
        "PLAYER_ONE_ID": player_one,
        "PLAYER_TWO_ID": player_two,
        "PLAYER_THREE_ID": player_three,
        "PLAYER_FOUR_ID": player_four,
        "PLAYER_FIVE_ID": player_five,
        "MINS": mins
    }
    
    return pd.DataFrame(out)
    

In [61]:
def get_lineup_shot_chart(lineup_id):
    data = shotchartlineupdetail.ShotChartLineupDetail(
        context_measure_detailed="FGA", 
        league_id="00", 
        season="2023-24", 
        season_type_all_star="Regular Season", 
        period=0, group_id=lineup_id
    )
    
    data = data.get_data_frames()
    data[0] = data[0].rename(columns={"GROUP_ID": "LINEUP_ID"})
    return data[0], data[1]

In [62]:
def get_all_team_ids():
    team_dict = teams.get_teams()
    return pd.json_normalize(team_dict)

In [63]:
def get_all_league_lineups():
    return pd.concat([get_all_team_lineups(team_id) for team_id in get_all_team_ids()["id"].tolist()])

In [70]:
def get_all_lineup_shot_charts():
    print("Getting all lineup IDs")
    lineup_ids = get_all_league_lineups()
    
    lineup_data = []
    league_avg = []
    counter = 1
    for id in lineup_ids["LINEUP_ID"]:
        print("Collecting data for lineup " + str(counter) + "/" + str(len(lineup_ids)))
        counter += 1
        time.sleep(.1)
        
        shot_data = get_lineup_shot_chart(id)
        lineup_data.append(shot_data[0])
        league_avg.append(shot_data[1])
    
    
    lineup_shot_chart_data = pd.concat(lineup_data)
    league_average_data = pd.concat(league_avg).drop_duplicates().reset_index(drop=True)
    
    #Get the individual player IDs into the shot chart dataframe
    lineup_shot_chart_data = pd.merge(lineup_shot_chart_data, lineup_ids, on="LINEUP_ID")
    
    league_fga = []
    league_fgm = []
    league_fg_perc = []
    
    counter = 1
    for index, row in lineup_shot_chart_data.iterrows():
        print("Finding league averages for shot " + str(counter) + "/" + len(lineup_shot_chart_data))
        counter += 1
        #Get league stats for each shot
        matches = lineup_average_data[lineup_average_data["SHOT_ZONE_BASIC"] == row["SHOT_ZONE_BASIC"]]
        matches = matches[matches["SHOT_ZONE_AREA"] == row["SHOT_ZONE_AREA"]]
        matches = matches[matches["SHOT_ZONE_RANGE"] == row["SHOT_ZONE_RANGE"]]
        
        
        league_fga.append(matches["FGA"].tolist()[0])
        league_fgm.append(matches["FGM"].tolist()[0])
        league_fg_perc.append(matches["FG_PCT"].tolist()[0])
    
    lineup_shot_chart_data["LEAGUE_FGA"] = league_fga
    lineup_shot_chart_data["LEAGUE_FGM"] = league_fgm
    lineup_shot_chart_data["LEAGUE_FG_PCT"] = league_fg_perc
    
    return lineup_shot_chart_data

In [71]:
data = get_all_lineup_shot_charts()

Getting all lineup IDs


KeyboardInterrupt: 